<a href="https://colab.research.google.com/github/AFNANAMIN/AI_Freelancing/blob/master/minvertexproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install dgl-cu100
#cuda deep learning library must study before deep diving into the code link:https://docs.dgl.ai/en/latest/tutorials/basics/1_first.html

     |████████████████████████████████| 19.6MB 444kB/s 


***Refrences of research papers i have followed:***
1.https://arxiv.org/pdf/1704.01665.p

In this paper  authors trained their neural network using a DQN algorithm and demonstrated the learned model’s ability to generalize to much larger problem instances than it was trained on. Their model generalized well even to instances of 1200 nodes (while being trained on instances of around 100 nodes), and could produce in 12 seconds solutions that were sometimes better than what a commercial solver could find in 1 hour. A big disadvantage of their method was that they used a “helper” function, to aid the neural network find better solutions. This helper function was a human designed one, and problem specific, which is what we would like to avoid.

2.https://arxiv.org/pdf/1803.08475.pdf

In this paper authors tackle several combinatorial optimization problems that involve routing agents on graphs, including our  familiar Traveling Salesman Problem. They treat the input as a graph and feed it to a modified Transformer architecture that embeds the nodes of the graph, and then sequentially chooses nodes to add to the tour until a full tour has been constructed. Treating the input as a graph is a more ‘correct’ approach than feeding it a sequence of nodes, since it eliminates the dependency on the order in which the cities are given in the input, as long their coordinates do not change. This means that however we permute the cities, the output of a given graph neural network will remain the same, unlike in the sequence approach.
In the architecture presented in the paper, the graph is embedded by a transformer style Encoder, which produces embeddings for all the nodes, and a single embedding vector for the entire graph. To produce the solution, a separate Decoder network is given each time a special context vector, that consists of the graph embedding and those of the last and first cities, and the embeddings of the unvisited cities, and it outputs a probability distribution on the unvisited cities, which is sampled to produce the next city to visit. The decoder sequentially produces cities until the tour is complete, and then a reward is given based on the length of the tour.

3.https://arxiv.org/pdf/1903.03332.pdf

In this paper the authors trained a Graph Convolutional Network to solve large instances of problems such as Minimum Vertex Cover (MVC) and Maximum Coverage Problem (MCP). They used a popular greedy algorithm for these problems to train the neural network to embed the graph and predict the next node to choose at each stage, and then further trained it using a DQN algorithm.They evaluated their method on graphs with millions of nodes, and achieved results that are both better and faster than current standard algorithms. While they did make use of a hand-crafted heuristic to help train their model, future works might do away with this constraint, and learn to solve huge problems Tabula Rasa.





In [0]:
class mean_val:
    def __init__(self):
        self.k = 0
        self.val = 0
        self.mean = 0
        
    def add(self,x):
        self.k += 1
        self.val += x
        self.mean = self.val/self.k
        
    def get(self):
        return self.mean
        
    
class logger:
    def __init__(self):
        self.log = dict()
        
    def add_log(self,name):
        self.log[name] = []
        
    def add_item(self,name,x):
        self.log[name].append(x)
        
    def get_log(self,name):
        return self.log[name]
    
    def get_current(self,name):
        return self.log[name][-1]
    
    def get_keys(self):
        return self.log.keys()
        

In [0]:
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F



msg = fn.copy_src(src='h', out='m')

def reduce(nodes):
    accum = torch.cat((torch.mean(nodes.mailbox['m'], 1),torch.max(nodes.mailbox['m'],1)[0]),dim=1)
    return {'hm': accum}

class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(3*in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(torch.cat((node.data['h'],node.data['hm']),dim=1))
        h = self.activation(h)
        return {'h' : h}

class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(msg, reduce)
        g.apply_nodes(func=self.apply_mod)
        g.ndata.pop('hm')
        return g.ndata.pop('h')


class ACNet(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(ACNet, self).__init__()
        
        self.policy = nn.Linear(hidden_dim,1)
        self.value = nn.Linear(hidden_dim,1)
        self.layers = nn.ModuleList([
            GCN(in_dim, hidden_dim, F.relu),
            GCN(hidden_dim, hidden_dim, F.relu),
            GCN(hidden_dim, hidden_dim, F.relu)])#3 hidden layers

    def forward(self, g):
        h = g.ndata['x']
        for conv in self.layers:
            h = conv(g, h)
        g.ndata['h'] = h
        mN = dgl.mean_nodes(g, 'h')
        PI = self.policy(g.ndata['h'])
        V = self.value(mN)
        g.ndata.pop('h')
        return PI, V

In [0]:
import torch #deep learning library for neural network
import numpy as np
import dgl
import torch.nn.functional as F
from copy import deepcopy as dc
#from Models import ACNet
#from Utils import stack_indices, stack_list_indices, max_graph_array
#from log_utils import mean_val, logger


class DiscreteActorCritic:#percepts are known thats why i take dicrete actor critic
    def __init__(self, problem, cuda_flag):
        self.problem = problem
        ndim = self.problem.get_graph_dims()
        if cuda_flag:
            self.model = ACNet(ndim,264,1).cuda()
        else:
            self.model = ACNet(ndim,264,1)
        self.gamma = 0.98
        self.optimizer = torch.optim.Adam(self.model.parameters(),lr=0.0003)
        self.batch_size = 32#in each iteration data size goes 32
        self.num_episodes = 1
        self.cuda = cuda_flag
        self.log = logger()
        self.log.add_log('tot_return')
        self.log.add_log('TD_error')
        self.log.add_log('entropy')
        
    def run_episode(self):
        sum_r = 0
        state, done = self.problem.reset()
        [idx1,idx2] = self.problem.get_ilegal_actions(state)
        t = 0
        while done == False:
            G = dc(state.g)
            if self.cuda:
                G.ndata['x'] = G.ndata['x'].cuda()
            [pi,val] = self.model(G)
            pi = pi.squeeze()
            pi[idx1] = -float('Inf')
            pi = F.softmax(pi,dim=0)
            dist = torch.distributions.categorical.Categorical(pi)
            action = dist.sample()
            
            new_state, reward, done = self.problem.step(dc(state),action)
            [idx1,idx2] = self.problem.get_ilegal_actions(new_state)
            state = dc(new_state)
            sum_r += reward
            if (t==0):
                PI = pi[action].unsqueeze(0)
                R = reward.unsqueeze(0)
                V = val.unsqueeze(0)
                t += 1
            else:
                PI = torch.cat([PI,pi[action].unsqueeze(0)],dim=0)
                R = torch.cat([R,reward.unsqueeze(0)],dim=0)
                V = torch.cat([V,val.unsqueeze(0)],dim=0)
        self.log.add_item('tot_return',sum_r.item())
        tot_return = R.sum().item()
        for i in range(R.shape[0] - 1):
            R[-2-i] = R[-2-i] + self.gamma*R[-1-i]
            
        return PI, R, V, tot_return
    
    
    def update_model(self,PI,R,V):
        self.optimizer.zero_grad()
        if self.cuda:
            R = R.cuda()
        A = R.squeeze() - V.squeeze().detach()
        L_policy = -(torch.log(PI)*A).mean()
        L_value = F.smooth_l1_loss(V.squeeze(), R.squeeze())
        L_entropy = -(PI*PI.log()).mean()
        L = L_policy + L_value - 0.1*L_entropy
        L.backward()
        self.optimizer.step()
        self.log.add_item('TD_error',L_value.detach().item())
        self.log.add_item('entropy',L_entropy.cpu().detach().item())
        
    
    def train(self):
        mean_return = 0
        for i in range(self.num_episodes):
            [pi,r,v,tot_return] = self.run_episode()
            mean_return = mean_return + tot_return
            if (i == 0):
                PI = pi
                R = r
                V = v
            else:
                PI = torch.cat([PI,pi],dim=0)
                R = torch.cat([R,r],dim=0)
                V = torch.cat([V,v],dim=0)
                
        mean_return = mean_return/self.num_episodes
        self.update_model(PI,R,V)
        return self.log
        

In [0]:
import dgl
import torch
import networkx as nx#for plotting of graph 
import numpy as np#for mathematical computation

class State:
    def __init__(self,g,visited):
        self.N = g.number_of_nodes()
        self.g = g
        self.visited = visited


def init_state(N,P):
    g = nx.fast_gnp_random_graph(N,P)
    g = dgl.DGLGraph(g)
    norm_card = torch.Tensor(np.array(g.in_degrees() + g.out_degrees())/g.number_of_nodes()).unsqueeze(-1)
    g.ndata['x'] = torch.cat((torch.zeros((N,1)),norm_card),dim=1)
    visited = torch.zeros((1,N)).squeeze()
    return g,visited

class MVC:#envoirment
    def __init__(self,N,P):
        self.N = N
        self.P = P
        [g,visited] = init_state(self.N,self.P)
        self.init_state = State(g,visited)
        
    def get_graph_dims(self):
        return 2
        
    def reset(self):
        [g,visited] = init_state(self.N,self.P)
        state = State(g,visited)
        done = False
        return state, done
    
    def reset_fixed(self):
        done = False
        return self.init_state, done
    
    def get_ilegal_actions(self,state):
        idx1 = (state.visited == 1.).nonzero()
        idx2 = (state.visited == 0.).nonzero()
        return idx1, idx2
    
    def step(self,state,action):
        done = False
        state.g.ndata['x'][action,0] = 1.0 - state.g.ndata['x'][action,0]
        state.visited[action.item()] = 1.0
        reward = torch.Tensor(np.array([-1])).squeeze()
        
        edge_visited = torch.cat((state.visited[state.g.edges()[0]].unsqueeze(-1),state.visited[state.g.edges()[1]].unsqueeze(-1)),dim=1).max(dim=1)[0]
        if edge_visited.mean().item() == 1.0:
            done = True
        return state, reward, done

In [0]:
# -*- coding: utf-8 -*-



#from discrete_actor_critic import DiscreteActorCritic
#from MVC import MVC
import matplotlib.pyplot as plt
#from smooth_signal import smooth
import numpy as np
import time
import torch

n = 30  # number of nodes
p = 0.15 # edge probability
env = MVC(n,p)
cuda_flag = True#for gpu
alg = DiscreteActorCritic(env,cuda_flag)


num_episodes = 4000
for i in range(num_episodes):
    T1 = time.time()
    log = alg.train()
    T2 = time.time()
    print('Epoch: {}. R: {}. TD error: {}. H: {}. T: {}'.format(i,np.round(log.get_current('tot_return'),2),np.round(log.get_current('TD_error'),3),np.round(log.get_current('entropy'),3),np.round(T2-T1,3)))
    

Y = np.asarray(log.get_log('tot_return'))
Y2 = smooth(Y)
x = np.linspace(0, len(Y), len(Y))#total return of graph is taken as energy consumption
fig2 = plt.figure()
ax2 = plt.axes()
ax2.plot(x, Y , Y2)
plt.xlabel('network density ')
plt.ylabel('energy consumption')

Y = np.asarray(log.get_log('TD_error'))#temporal refrence learning error
Y2 = smooth(Y)
x = np.linspace(0, len(Y), len(Y))
fig2 = plt.figure()
ax2 = plt.axes()
ax2.plot(x, Y , Y2)
plt.xlabel('network density')#The error function reports back the difference between the estimated reward at any given state or time step and the actual reward received
plt.ylabel('link broken')

Y = np.asarray(log.get_log('entropy'))
Y2 = smooth(Y)
x = np.linspace(0, len(Y), len(Y))
fig2 = plt.figure()
ax2 = plt.axes()
ax2.plot(x, Y , Y2)
plt.xlabel('network density')
plt.ylabel('network lifetime')#entropy can be taken as network lifetime graph

PATH = 'mvc_net.pt'
torch.save(alg.model.state_dict(),PATH)


Epoch: 0. R: -29.0. TD error: 12.062. H: 0.199. T: 1.049
Epoch: 1. R: -28.0. TD error: 11.618. H: 0.193. T: 0.761
Epoch: 2. R: -28.0. TD error: 11.518. H: 0.194. T: 0.723
Epoch: 3. R: -27.0. TD error: 11.054. H: 0.187. T: 0.744
Epoch: 4. R: -28.0. TD error: 11.3. H: 0.193. T: 0.658
Epoch: 5. R: -27.0. TD error: 10.83. H: 0.187. T: 0.652
Epoch: 6. R: -28.0. TD error: 11.046. H: 0.193. T: 0.653
Epoch: 7. R: -24.0. TD error: 9.444. H: 0.171. T: 0.622
Epoch: 8. R: -27.0. TD error: 10.288. H: 0.187. T: 0.753
Epoch: 9. R: -26.0. TD error: 9.725. H: 0.181. T: 0.606
Epoch: 10. R: -28.0. TD error: 10.156. H: 0.193. T: 0.734
Epoch: 11. R: -29.0. TD error: 10.319. H: 0.199. T: 0.614
Epoch: 12. R: -28.0. TD error: 9.69. H: 0.193. T: 0.711
Epoch: 13. R: -27.0. TD error: 9.057. H: 0.188. T: 0.665
Epoch: 14. R: -26.0. TD error: 8.429. H: 0.182. T: 0.673
Epoch: 15. R: -28.0. TD error: 8.651. H: 0.191. T: 0.77
Epoch: 16. R: -27.0. TD error: 8.07. H: 0.188. T: 0.714
Epoch: 17. R: -28.0. TD error: 8.101.

In [0]:
# -*- coding: utf-8 -*-


import torch
#from MVC import MVC
import dgl

import torch.nn.functional as F
#from Models import ACNet
import time
from copy import deepcopy as dc

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np



cuda_flag = True
num_nodes = 30
p_edge = 0.15
mvc = MVC(num_nodes,p_edge)
ndim = mvc.get_graph_dims()

if cuda_flag:
    NN = ACNet(ndim,264,1).cuda()#output node 1 hidden nodes 264 and dimension of graph is input nodes
else:
    NN = ACNet(ndim,264,1)
PATH = 'mvc_net.pt'
NN.load_state_dict(torch.load(PATH))

init_state,done = mvc.reset()
pos = nx.spring_layout(init_state.g.to_networkx(), iterations=20)

#### GCN Policy
state = dc(init_state)
if cuda_flag:
    state.g.ndata['x'] = state.g.ndata['x'].cuda()
sum_r = 0
T1 = time.time()
[idx1,idx2] = mvc.get_ilegal_actions(state)
while done == False:
    G = state.g
    [pi,val] = NN(G)
    pi = pi.squeeze()
    pi[idx1] = -float('Inf')
    pi = F.softmax(pi,dim=0)
    dist = torch.distributions.categorical.Categorical(pi)
    action = dist.sample()            
    new_state, reward, done = mvc.step(state,action)
    [idx1,idx2] = mvc.get_ilegal_actions(new_state)
    state = new_state
    sum_r += reward
T2 = time.time()

node_tag = state.g.ndata['x'][:,0].cpu().squeeze().numpy().tolist()
nx.draw(state.g.to_networkx(), pos, node_color=node_tag, with_labels=True)
plt.show()



### Heuristic Policy
state = dc(init_state)
done = False
sum_r2 = 0
T1 = time.time()
[idx1,idx2] = mvc.get_ilegal_actions(state)
while done == False:
    G = state.g
    un_allowed = idx1.numpy().squeeze()
    degree = G.in_degrees() + G.out_degrees()
    degree[un_allowed] = 0
    degree = torch.Tensor(np.array(degree))
    action = degree.argmax()
    
    new_state, reward, done = mvc.step(state,action)
    [idx1,idx2] = mvc.get_ilegal_actions(new_state)
    state = new_state
    sum_r2 += reward
T2 = time.time()

node_tag = state.g.ndata['x'][:,0].cpu().squeeze().numpy().tolist()
nx.draw(state.g.to_networkx(), pos, node_color=node_tag, with_labels=True)
plt.show()
plt.savefig('my_figure.png')

print('Ratio: {}'.format((sum_r/sum_r2).item()))
